In [1]:
from autofeat.dataset import *

source_dataset = KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance")

In [2]:
from autofeat.transform import *

transform = (
    Cast()
    .then(Encode())
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [3]:
derived_dataset = source_dataset.derive(transform)

In [4]:
features = derived_dataset.features(
    Filter(eq={"EmployeeID": "51A4-EA6E"}),
    Filter(eq={"EmployeeID": "A4A4-F49C"}),
)

Dataset URL: https://www.kaggle.com/datasets/mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance


In [5]:
features.collect()

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from PerformanceRating.csv,max(WorkLifeBalance) from PerformanceRating.csv,mean(WorkLifeBalance) from PerformanceRating.csv,median(WorkLifeBalance) from PerformanceRating.csv,min(WorkLifeBalance) from PerformanceRating.csv,std(WorkLifeBalance) from PerformanceRating.csv,sum(WorkLifeBalance) from PerformanceRating.csv,var(WorkLifeBalance) from PerformanceRating.csv,max(JobSatisfaction) from PerformanceRating.csv,mean(JobSatisfaction) from PerformanceRating.csv,median(JobSatisfaction) from PerformanceRating.csv,min(JobSatisfaction) from PerformanceRating.csv,std(JobSatisfaction) from PerformanceRating.csv,sum(JobSatisfaction) from PerformanceRating.csv,var(JobSatisfaction) from PerformanceRating.csv,max(EnvironmentSatisfaction) from PerformanceRating.csv,mean(EnvironmentSatisfaction) from PerformanceRating.csv,median(EnvironmentSatisfaction) from PerformanceRating.csv,min(EnvironmentSatisfaction) from PerformanceRating.csv,std(EnvironmentSatisfaction) from PerformanceRating.csv,sum(EnvironmentSatisfaction) from PerformanceRating.csv,var(EnvironmentSatisfaction) from PerformanceRating.csv,max(RelationshipSatisfaction) from PerformanceRating.csv,mean(RelationshipSatisfaction) from PerformanceRating.csv,median(RelationshipSatisfaction) from PerformanceRating.csv,min(RelationshipSatisfaction) from PerformanceRating.csv,std(RelationshipSatisfaction) from PerformanceRating.csv,sum(RelationshipSatisfaction) from PerformanceRating.csv,var(RelationshipSatisfaction) from PerformanceRating.csv,…,max(StockOptionLevel) / std(Education) from Employee.csv,min(YearsInMostRecentRole) + max(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) - max(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * max(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) / max(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) / min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) + std(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) - std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * std(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) / std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) / min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) + max(StockOptionLevel) from Employee.csv,min(YearsInMostRecentRole) - max(StockOptionLevel) from Employee.csv,max(StockOptionLevel) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * max(StockOptionLevel) from Employee.csv,min(YearsInMostRecentRole) / max(StockOptionLevel) from Employee.csv,max(StockOptionLevel) / min(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) + std(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) - std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) - max(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) * std(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) / std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) / max(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) + max(StockOptionLevel) from Employee.csv,max(YearsInMostRecentRole) - max(StockOptionLevel) from Employee.csv,max(StockOptionLevel) - max(YearsInMostRecentRole) from Employee.csv,max(YearsInMostRecentRole) * max(StockOptionLevel) from Employee.csv,max(YearsInMostRecentRole) / max(StockOptionLevel) from Employee.csv,max(StockOptionLevel) / max(YearsInMostRecentRole) from 

In [24]:
import numpy
import sklearn.feature_selection
import xgboost

examples = [
    (
        Filter(eq={"EmployeeID": "51A4-EA6E"}),
        True,
    ),
    (
        Filter(eq={"EmployeeID": "A4A4-F49C"}),
        False,
    ),
]

features = (
    derived_dataset
    .features(filter for filter, _ in examples)
    .collect()
)

selector = sklearn.feature_selection.SelectFromModel(
    xgboost.XGBClassifier(missing=float("inf")),
    max_features=10,
)

selector.fit(
    X=features.to_numpy(),
    y=numpy.array([label for _, label in examples]),
)

selection = [
    features.columns[i]
    for i, selected in enumerate(selector.get_support())
    if selected
]

In [25]:
selection


['count(*) from RatingLevel.csv',
 'max(RatingID) from RatingLevel.csv',
 'mean(RatingID) from RatingLevel.csv',
 'median(RatingID) from RatingLevel.csv',
 'min(RatingID) from RatingLevel.csv',
 'std(RatingID) from RatingLevel.csv',
 'sum(RatingID) from RatingLevel.csv',
 'var(RatingID) from RatingLevel.csv',
 'count(*) from PerformanceRating.csv',
 'max(WorkLifeBalance) from PerformanceRating.csv']